# Notebook Markdown Section
# ETL – Transform Phase for Instagram Analytics

This section focuses on the **Transform** step of the ETL pipeline, where cleaned Instagram data is structured into **dimension and fact tables** suitable for analysis and Business Intelligence reporting.

---

### 1. Time Dimension (`time_dim`)
- Extracts temporal information from the `upload_date`.
- Adds:
  - `year`
  - `month`
  - `day`
- Each unique date in the dataset becomes a row in the time dimension.

---

### 2. Content Dimension (`content_dim`)
- Captures characteristics of Instagram posts:
  - `media_type` (e.g., Photo, Video)
  - `content_category` (e.g., Lifestyle, Food)
  - `caption_length`
  - `hashtags_count`
- Assigns a unique `content_id` to each distinct combination of attributes.
- Serves as a **lookup table** for the fact table.

---

### 3. Fact Table (`instagram_fact`)
- Combines the cleaned data with the content dimension.
- Includes:
  - Post metrics: `likes`, `comments`, `shares`, `saves`
  - Reach and engagement: `reach`, `impressions`, `engagement_rate`
  - Followers gained and traffic source
  - Foreign key: `content_id` linking to the content dimension
- Each row corresponds to a single Instagram post and its performance metrics.

---

### 4. Output
- Saves transformed tables as CSV files in the `data/` folder:
  - `time_dim.csv`
  - `content_dim.csv`
  - `instagram_fact.csv`

---

This structure allows for efficient **BI queries**, analytics, and dashboard creation by separating **dimensions** (time, content) from **facts** (post performance metrics).









In [8]:
# Transform_Data
import pandas as pd
import os

processed_dir = '../data/processed'
os.makedirs(processed_dir, exist_ok=True)
def clean_data():
    # Load raw data
    df = pd.read_csv("../data/staging/Instagram_Analytics.csv")
    print("Initial dataset shape:", df.shape)
    # ---- Missing Values ----
    print("Missing values before cleaning:")
    print(df.isnull().sum())
    # Categorical columns → mode
    categorical_cols = ['media_type', 'traffic_source', 'content_category']
    for col in categorical_cols:
        df[col] = df[col].fillna(df[col].mode()[0])
    # Numerical columns → median
    numerical_cols = [
        'likes', 'comments', 'shares', 'saves',
        'reach', 'impressions', 'followers_gained',
        'caption_length', 'hashtags_count', 'engagement_rate']
    for col in numerical_cols:
        df[col] = df[col].fillna(df[col].median())
    # ---- Data Type Fixing ----
    df['upload_date'] = pd.to_datetime(df['upload_date'], errors='coerce')
    # Remove rows with invalid dates
    df = df.dropna(subset=['upload_date'])
    # ---- Standardization ----
    df['media_type'] = df['media_type'].str.title().str.strip()
    df['traffic_source'] = df['traffic_source'].str.title().str.strip()
    df['content_category'] = df['content_category'].str.title().str.strip()
    # ---- Remove Duplicates ----
    df = df.drop_duplicates(subset=['post_id'])
    print("Missing values after cleaning:")
    print(df.isnull().sum())
    import os
    print(os.getcwd())
    # Save cleaned data
    df.to_csv(f'{processed_dir}/Instagram_Analytics_clean.csv', index=False)
    print("Cleaned data saved")
def transform_data():
    df = pd.read_csv(f'{processed_dir}/Instagram_Analytics_clean.csv')
    # ---- Time Dimension ----
    time_dim = df[['upload_date']].drop_duplicates()
    time_dim['year'] = pd.to_datetime(time_dim['upload_date']).dt.year
    time_dim['month'] = pd.to_datetime(time_dim['upload_date']).dt.month
    time_dim['day'] = pd.to_datetime(time_dim['upload_date']).dt.day
     # ---- Content Dimension ----
    content_dim = df[[
        'media_type', 'content_category',
        'caption_length', 'hashtags_count'
    ]].drop_duplicates().reset_index(drop=True)
    content_dim['content_id'] = content_dim.index + 1

    # ---- Fact Table ----
    instagram_fact = df.merge(
        content_dim,
        on=['media_type', 'content_category', 'caption_length', 'hashtags_count'],
        how='left'
    )

    instagram_fact = instagram_fact[[
        'post_id',
        'upload_date',
        'content_id',
        'likes', 'comments', 'shares', 'saves',
        'reach', 'impressions',
        'followers_gained',
        'engagement_rate',
        'traffic_source']]
 # ---- Save Transformed Data ----
    time_dim.to_csv(f'{processed_dir}/time_dim.csv', index=False)
    content_dim.to_csv(f'{processed_dir}/content_dim.csv', index=False)
    instagram_fact.to_csv(f'{processed_dir}/instagram_fact.csv', index=False)

    print("Transformation completed.")
def validate_data():
    time_dim = pd.read_csv(f'{processed_dir}/time_dim.csv')
    content_dim = pd.read_csv(f'{processed_dir}/content_dim.csv')
    fact = pd.read_csv(f'{processed_dir}/instagram_fact.csv')

    # ---- Missing Values ----
    assert time_dim.isnull().sum().sum() == 0, "Missing values in Time_Dim"
    assert content_dim.isnull().sum().sum() == 0, "Missing values in Content_Dim"
    assert fact.isnull().sum().sum() == 0, "Missing values in Fact table"

    # ---- Key Integrity ----
    assert fact['content_id'].isin(content_dim['content_id']).all(), \
        "Invalid content_id in fact table"

    # ---- Numeric Validation ----
    numeric_checks = ['likes', 'comments', 'shares', 'saves', 'reach', 'impressions']
    for col in numeric_checks:
        assert (fact[col] >= 0).all(), f"Negative values found in {col}"
    print("Data validation completed successfully.")
def main():
    clean_data()
    transform_data()
    validate_data()
if __name__ == "__main__":
    main()





    
    

    

    


    


    
                                       

    

    



Initial dataset shape: (29999, 15)
Missing values before cleaning:
post_id             0
upload_date         0
media_type          0
likes               0
comments            0
shares              0
saves               0
reach               0
impressions         0
caption_length      0
hashtags_count      0
followers_gained    0
traffic_source      0
engagement_rate     0
content_category    0
dtype: int64
Missing values after cleaning:
post_id             0
upload_date         0
media_type          0
likes               0
comments            0
shares              0
saves               0
reach               0
impressions         0
caption_length      0
hashtags_count      0
followers_gained    0
traffic_source      0
engagement_rate     0
content_category    0
dtype: int64
C:\Users\malek\OneDrive\Desktop\BI Project\notebooks
Cleaned data saved
Transformation completed.
Data validation completed successfully.
